In [52]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score

# Load data

### Initialize path constant

In [53]:
DATA_PATH = '../data'
RAW_DATA_PATH = DATA_PATH+'/raw'
PROCESSED_DATA_PATH = DATA_PATH+'/processed'

### Read processed data from CSV

In [54]:
data = pd.read_csv('{}/processed_data.csv'.format(PROCESSED_DATA_PATH))
X = data.drop(['heart_disease_diagnosis','sex'], axis=1)
y = data['heart_disease_diagnosis']

# Cross Validation

### Train-test split

In [55]:
X_train, X_test, y_train, y_test = train_test_split(X.values,y.values, test_size=0.33)

# Modelling

#### Print metrics function

In [56]:
def print_metrics(y_true, y_pred):
    accuracy = accuracy_score(y_pred=y_pred, y_true=y_test)
    precision = precision_score(y_pred=y_pred, y_true=y_test, average='macro')
    recall = recall_score(y_pred=y_pred, y_true=y_test, average='macro')

    print("Accuracy = {}".format(accuracy))
    print("Precision = {}".format(precision))
    print("Recall = {}".format(recall))

### Random Forest Classifier

In [57]:
rfc = RandomForestClassifier()
rfc.fit(X_train,y_train)
y_pred = rfc.predict(X_test)

print_metrics(y_test,y_pred)

Accuracy = 0.5
Precision = 0.32692726663314897
Recall = 0.3166284196844468


/home/adylan/.local/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/adylan/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### Gradient Boosting Classifier

In [59]:
gbc = GradientBoostingClassifier()
gbc.fit(X_train,y_train)
y_pred = gbc.predict(X_test)

print_metrics(y_test,y_pred)

Accuracy = 0.5426356589147286
Precision = 0.3320552836621531
Recall = 0.3353964937156788
